# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
# Bibliotecas essenciais
import numpy as np
import pandas as pd

# Bibliotecas de aprendizado de máquina
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

# Bibliotecas de estatística
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [2]:
df = pd.read_csv("./input/previsao_de_renda.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

### 1°

In [4]:
df.drop(columns=["Unnamed: 0", "data_ref", "id_cliente"], inplace=True)

In [5]:
df.isna().any(axis=1).value_counts()

False    12427
True      2573
Name: count, dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
x = df.drop(columns=["renda"])
y = df["renda"]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=100
)

### 2°

In [8]:
lista_de_alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
dicionario_de_sumarios = {}
dicionario_treinos_ridge = {}
dicionario_testes_ridge = {}
dicionario_treinos_lasso = {}
dicionario_testes_lasso = {}

In [9]:
df.columns

Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [10]:
modelo = "renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia"
md_teste = smf.ols(modelo, data=x_train.join(y_train))

for alpha in lista_de_alpha:
    reg = md_teste.fit_regularized(
        method="elastic_net", refit=False, L1_wt=0, alpha=alpha
    )

    y_pred_test = reg.predict(x_test)

    # Cálculo de R-Quadrado
    tss_test = ((y_test - y_test.mean()) ** 2).sum()
    rss_test = ((y_test - y_pred_test) ** 2).sum()
    r_quadrado_test = 1 - rss_test / tss_test

    # Cálculo de R-Quadrado Ajustado
    qtd_variaveis_explicativas = len(reg.params) - 1
    r_quadrado_ajustado_teste = 1 - (1 - r_quadrado_test) * (len(y_test) - 1) / (
        len(y_test) - qtd_variaveis_explicativas - 1
    )

    # Cálculo de AIC
    residuo_quadrado_test = rss_test / len(y_test)
    log_vero_test = (
        -len(y_test) / 2 * (np.log(2 * np.pi) + np.log(residuo_quadrado_test) + 1)
    )
    aic = 2 * len(reg.params) - 2 * log_vero_test

    dicionario_testes_ridge[alpha] = {
        "L2 R-Quadrado Teste": round(r_quadrado_test, 3),
        "L2 R-Quadrado Ajustado Teste": round(r_quadrado_ajustado_teste, 3),
        "L2 AIC Teste": aic,
        "L2 Coef Teste": reg.params.tolist(),
    }

In [11]:
md_treino = smf.ols(modelo, data=x_train.join(y_train))

for alpha in lista_de_alpha:
    reg = md_treino.fit_regularized(
        method="elastic_net", refit=False, L1_wt=0, alpha=alpha
    )
    y_pred_train = reg.predict(x_train)

    # Cálculo de R-Quadrado
    tss_train = ((y_train - y_train.mean()) ** 2).sum()
    rss_train = ((y_train - y_pred_train) ** 2).sum()
    r_quadrado_train = 1 - rss_train / tss_train

    # Cálculo de R-Quadrado Ajustado
    qtd_variaveis_explicativas = len(reg.params) - 1
    r_quadrado_ajustado_train = 1 - (1 - r_quadrado_train) * (len(y_train) - 1) / (
        len(y_train) - qtd_variaveis_explicativas - 1
    )

    # Cálculo de AIC
    residuo_quadrado_train = rss_train / len(y_train)
    log_vero_train = (
        -len(y_train) / 2 * (np.log(2 * np.pi) + np.log(residuo_quadrado_train) + 1)
    )
    aic = 2 * len(reg.params) - 2 * log_vero_train

    dicionario_treinos_ridge[alpha] = {
        "L2 R-Quadrado Treino": round(r_quadrado_train, 3),
        "L2 R-Quadrado Ajustado Treino": round(r_quadrado_ajustado_train, 3),
        "L2 AIC Treino": aic,
        "L2 Coef Treino": reg.params.tolist(),
    }

In [12]:
df_info = pd.DataFrame(dicionario_treinos_ridge).T.join(
    pd.DataFrame(dicionario_testes_ridge).T
)
ordem_variaveis_ridge = [
    "L2 R-Quadrado Treino",
    "L2 R-Quadrado Teste",
    "L2 R-Quadrado Ajustado Treino",
    "L2 R-Quadrado Ajustado Teste",
    "L2 AIC Treino",
    "L2 AIC Teste",
    "L2 Coef Treino",
    "L2 Coef Teste",
]

df_info = df_info[ordem_variaveis_ridge]
df_info.reset_index()

,index,L2 R-Quadrado Treino,L2 R-Quadrado Teste,L2 R-Quadrado Ajustado Treino,L2 R-Quadrado Ajustado Teste,L2 AIC Treino,L2 AIC Teste,L2 Coef Treino,L2 Coef Teste
0,0.000,0.247,0.293,0.245,0.287,193974.68557,63565.211548,"[-5588.4314396625805, 6096.61432147802, 131.43...","[-5588.4314396625805, 6096.61432147802, 131.43..."
1,0.001,0.246,0.293,0.244,0.287,193976.249276,63565.194701,"[-2679.724846762771, 6053.47976770689, 137.860...","[-2679.724846762771, 6053.47976770689, 137.860..."
2,0.005,0.246,0.292,0.244,0.287,193979.053038,63565.619005,"[-1484.805359441785, 5923.742484634264, 168.20...","[-1484.805359441785, 5923.742484634264, 168.20..."
3,0.010,0.246,0.292,0.244,0.287,193982.188798,63566.755976,"[-1168.5155872449704, 5774.648429427202, 203.6...","[-1168.5155872449704, 5774.648429427202, 203.6..."
4,0.050,0.242,0.288,0.24,0.282,194030.32955,63586.874349,"[-543.9697432662743, 4838.329457461772, 400.74...","[-543.9697432662743, 4838.329457461772, 400.74..."
5,0.100,0.236,0.28,0.234,0.274,194109.512731,63619.866697,"[-304.9487956371316, 4054.4457907166857, 514.7...","[-304.9487956371316, 4054.4457907166857, 514.7..."


### 3°

In [13]:
modelo = "renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia"
md_teste = smf.ols(modelo, data=x_train.join(y_train))

for alpha in lista_de_alpha:
    reg = md_teste.fit_regularized(
        method="elastic_net", refit=True, L1_wt=1, alpha=alpha
    )

    y_pred_test = reg.predict(x_test)

    # Cálculo de R-Quadrado
    tss_test = ((y_test - y_test.mean()) ** 2).sum()
    rss_test = ((y_test - y_pred_test) ** 2).sum()
    r_quadrado_test = 1 - rss_test / tss_test

    # Cálculo de R-Quadrado Ajustado
    qtd_variaveis_explicativas = len(reg.params) - 1
    r_quadrado_ajustado_teste = 1 - (1 - r_quadrado_test) * (len(y_test) - 1) / (
        len(y_test) - qtd_variaveis_explicativas - 1
    )

    # Cálculo de AIC
    residuo_quadrado_test = rss_test / len(y_test)
    log_vero_test = (
        -len(y_test) / 2 * (np.log(2 * np.pi) + np.log(residuo_quadrado_test) + 1)
    )
    aic = 2 * len(reg.params) - 2 * log_vero_test

    dicionario_testes_lasso[alpha] = {
        "L1 R-Quadrado Teste": round(r_quadrado_test, 3),
        "L1 R-Quadrado Ajustado Teste": round(r_quadrado_ajustado_teste, 3),
        "L1 AIC Teste": aic,
        "L1 Coef Teste": reg.params.tolist(),
    }

In [14]:
md_treino = smf.ols(modelo, data=x_train.join(y_train))

for alpha in lista_de_alpha:
    reg = md_treino.fit_regularized(
        method="elastic_net", refit=True, L1_wt=1, alpha=alpha
    )
    y_pred_train = reg.predict(x_train)

    # Cálculo de R-Quadrado
    tss_train = ((y_train - y_train.mean()) ** 2).sum()
    rss_train = ((y_train - y_pred_train) ** 2).sum()
    r_quadrado_train = 1 - rss_train / tss_train

    # Cálculo de R-Quadrado Ajustado
    qtd_variaveis_explicativas = len(reg.params) - 1
    r_quadrado_ajustado_train = 1 - (1 - r_quadrado_train) * (len(y_train) - 1) / (
        len(y_train) - qtd_variaveis_explicativas - 1
    )

    # Cálculo de AIC
    residuo_quadrado_train = rss_train / len(y_train)
    log_vero_train = (
        -len(y_train) / 2 * (np.log(2 * np.pi) + np.log(residuo_quadrado_train) + 1)
    )
    aic = 2 * len(reg.params) - 2 * log_vero_train

    dicionario_treinos_lasso[alpha] = {
        "L1 R-Quadrado Treino": round(r_quadrado_train, 3),
        "L1 R-Quadrado Ajustado Treino": round(r_quadrado_ajustado_train, 3),
        "L1 AIC Treino": aic,
        "L1 Coef Treino": reg.params.tolist(),
    }

    dicionario_de_sumarios[alpha] = reg.summary()

In [15]:
ordem_variaveis_lasso = [
    "L1 R-Quadrado Treino",
    "L1 R-Quadrado Teste",
    "L1 R-Quadrado Ajustado Treino",
    "L1 R-Quadrado Ajustado Teste",
    "L1 AIC Treino",
    "L1 AIC Teste",
    "L1 Coef Treino",
    "L1 Coef Teste",
]

In [16]:
df_info = df_info.join(pd.DataFrame(dicionario_treinos_lasso).T)
df_info = df_info.join(pd.DataFrame(dicionario_testes_lasso).T)
df_info = df_info[ordem_variaveis_ridge + ordem_variaveis_lasso]
df_info = df_info.T

In [17]:
dicionario_de_sumarios[0.050]

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Thu, 09 Jan 2025   Prob (F-statistic):               0.00
Time:                        20:53:23   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -5588.4314   2718.256     -2.056      0.040   -1.09e+04    -260.053
sexo[T.M]                         6096.6143    187.396     32.533      0.000    5729.276    6463.953
posse_de_veiculo[T.True]           131.4369    181.085      0.726      0.468    -223.530     486.404
posse_de_imovel[T.True]            300.1032    178.199      1.684      0.092     -49.206     649.412
tipo_renda[T.Bolsista]           -1449.1393   4001.765     -0.362      0.717   -9293.476    6395.198
tipo_renda[T.Empresário]           735.6595    191.004      3.852      0.000     361.249    1110.069
tipo_renda[T.Pensionista]        -2181.9763   2834.454     -0.770      0.441   -7738.127    3374.174
tipo_renda[T.Servidor público]     -79.8440    284.539     -0.281      0.779    -637.604     477.915
educacao[T.Pós graduação]         1595.3814   1928.847      0.827      0.408   -2185.582    5376.344
educacao[T.Secundário]             995.9173    900.140      1.106      0.269    -768.554    2760.388
educacao[T.Superior completo]     1409.2209    903.900      1.559      0.119    -362.622    3181.064
educacao[T.Superior incompleto]    705.8989    973.085      0.725      0.468   -1201.561    2613.359
estado_civil[T.Separado]          1099.3182   1269.627      0.866      0.387   -1389.429    3588.065
estado_civil[T.Solteiro]           694.4665   1239.480      0.560      0.575   -1735.186    3124.119
estado_civil[T.União]             -383.9909    319.330     -1.202      0.229   -1009.949     241.967
estado_civil[T.Viúvo]             1013.2650   1335.786      0.759      0.448   -1605.168    3631.698
tipo_residencia[T.Casa]           -395.7604    695.705     -0.569      0.569   -1759.494     967.973
tipo_residencia[T.Com os pais]    -428.0415    773.169     -0.554      0.580   -1943.623    1087.540
tipo_residencia[T.Comunitário]    -566.1160   1371.132     -0.413      0.680   -3253.835    2121.603
tipo_residencia[T.Estúdio]         370.6936   1317.252      0.281      0.778   -2211.409    2952.796
tipo_residencia[T.Governamental]  -259.5952    847.102     -0.306      0.759   -1920.101    1400.910
qtd_filhos                       -1197.9442   1223.286     -0.979      0.327   -3595.854    1199.965
idade                               40.2946     10.193      3.953      0.000      20.314      60.276
tempo_emprego                      566.2190     13.231     42.794      0.000     540.283     592.155
qt_pessoas_residencia             1188.1652   1221.022      0.973      0.331   -1205.306    3581.636
==============================================================================
Omnibus:                    13662.596   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8593055.220
Skew:             

In [18]:
df_info

,0.000,0.001,0.005,0.010,0.050,0.100
L2 R-Quadrado Treino,0.247,0.246,0.246,0.246,0.242,0.236
L2 R-Quadrado Teste,0.293,0.293,0.292,0.292,0.288,0.28
L2 R-Quadrado Ajustado Treino,0.245,0.244,0.244,0.244,0.24,0.234
L2 R-Quadrado Ajustado Teste,0.287,0.287,0.287,0.287,0.282,0.274
L2 AIC Treino,193974.68557,193976.249276,193979.053038,193982.188798,194030.32955,194109.512731
L2 AIC Teste,63565.211548,63565.194701,63565.619005,63566.755976,63586.874349,63619.866697
L2 Coef Treino,"[-5588.4314396625805, 6096.61432147802, 131.43...","[-2679.724846762771, 6053.47976770689, 137.860...","[-1484.805359441785, 5923.742484634264, 168.20...","[-1168.5155872449704, 5774.648429427202, 203.6...","[-543.9697432662743, 4838.329457461772, 400.74...","[-304.9487956371316, 4054.4457907166857, 514.7..."
L2 Coef Teste,"[-5588.4314396625805, 6096.61432147802, 131.43...","[-2679.724846762771, 6053.47976770689, 137.860...","[-1484.805359441785, 5923.742484634264, 168.20...","[-1168.5155872449704, 5774.648429427202, 203.6...","[-543.9697432662743, 4838.329457461772, 400.74...","[-304.9487956371316, 4054.4457907166857, 514.7..."
L1 R-Quadrado Treino,0.247,0.247,0.247,0.247,0.247,0.247
L1 R-Quadrado Teste,0.293,0.293,0.293,0.293,0.293,0.293


Resposta: De todos os modelos gerados, o tipo de regularização que constantemente apresenta melhores indicadores é a Lasso, dos gerados por Ridge o único que rivaliza com os melhores indicadores de "R-Quadrado" e "R-Quadrado Ajustado" é o de alpha=0.001. Por uma maior facilidade de trabalho e melhores resultados o modelo selecionado é Lasso com alpha=0.001. A facilidade de se trabalhar com Lasso é evidente pela possibilidade de utilizar o método reg.summary() para mapear todos os valores do modelo, o que para os modelos regulados por ridge foi necessário programar as fórmulas para obter os respectivos valores.

### 4°

In [19]:
pd.get_dummies(x_train)

,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_F,sexo_M,tipo_renda_Assalariado,tipo_renda_Bolsista,...,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Aluguel,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
3889,True,False,0,50,3.441096,2.0,False,True,True,False,...,False,False,False,False,False,False,False,True,False,False
14483,False,False,0,52,2.745205,1.0,True,False,True,False,...,True,False,False,False,False,True,False,False,False,False
762,True,False,0,38,6.539726,2.0,True,False,True,False,...,False,False,False,False,False,True,False,False,False,False
3078,True,True,0,27,7.624658,2.0,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
10423,False,False,0,47,4.550685,2.0,True,False,True,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,True,True,1,37,0.265753,3.0,False,True,True,False,...,False,False,False,False,False,True,False,False,False,False
14610,True,True,1,41,4.016438,3.0,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
9679,False,True,2,36,18.043836,4.0,True,False,True,False,...,False,False,False,False,False,True,False,False,False,False
8373,False,True,0,37,2.701370,2.0,False,True,True,False,...,False,False,False,False,False,True,False,False,False,False


In [20]:
X_train = pd.get_dummies(x_train, drop_first=True)
X_test = pd.get_dummies(x_test, drop_first=True)


X_train.columns = X_train.columns.str.replace(" ", "_", regex=True)
X_test.columns = X_test.columns.str.replace(" ", "_", regex=True)

y = y_train

X_train = X_train.astype(float)
X_test = X_test.astype(float)
y = y.astype(float)


def stepwise_selection(
    X_train, y, initial_list=[], threshold_in=0.05, threshold_out=0.05, verbose=True
):
    """Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed = False
        # forward step
        excluded = list(set(X_train.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype("float64"))
        for new_column in excluded:
            model = sm.OLS(
                y, sm.add_constant(pd.DataFrame(X_train[included + [new_column]]))
            ).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed = True
            if verbose:
                print("Add  {:30} with p-value {:.6}".format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X_train[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print("Drop {:30} with p-value {:.6}".format(worst_feature, worst_pval))
        if not changed:
            break
    return included


variaveis = stepwise_selection(X_train, y)

print("resulting features:")
print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 2.85907e-248
#############
['tempo_emprego', 'sexo_M']
Add  idade                          with p-value 6.86247e-06
#############
['tempo_emprego', 'sexo_M', 'idade']
Add  tipo_renda_Empresário          with p-value 3.02238e-05
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário']
Add  educacao_Superior_completo     with p-value 0.00798815
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário', 'educacao_Superior_completo']
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário', 'educacao_Superior_completo']
resulting features:
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário', 'educacao_Superior_completo']


In [21]:
variaveis
string_variaveis = "tempo_emprego + sexo_M + idade + tipo_renda_Empresário + educacao_Superior_completo"
lista_variaveis = variaveis
X_test = X_test[lista_variaveis]

In [22]:
X_train = X_train[lista_variaveis]
X_train.columns

Index(['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário',
       'educacao_Superior_completo'],
      dtype='object')

In [23]:
dicionario_indicadores = {}

for alpha in lista_de_alpha:
    md_treino = smf.ols(f'renda ~ {string_variaveis}', data=X_train.join(y_train))
    reg = md_treino.fit_regularized(method="elastic_net", refit=True, L1_wt=1, alpha=alpha)

    y_pred_test = reg.predict(X_test)

    r_quadrado = r2_score(y_test, y_pred_test)
    r_quadrado_ajustado = 1 - ((1 - r_quadrado) * (len(y_test) - 1)) / (len(y_test) - len(X_test.columns) - 1)
    
    rss = np.power(y_test - y_pred_test, 2).sum()
    log_vero_test = (
    -len(y_test) / 2 * (np.log(2 * np.pi) + np.log(rss/len(y_test)) + 1)
    )
    
    
    aic = 2* len(reg.params) - 2 * log_vero_test
    
    dicionario_indicadores[alpha] = {'R-Quadrado': round(r_quadrado, 3), 'R-Quadrado Ajustado': round(r_quadrado_ajustado, 3), 'AIC': aic, 'Coef': reg.params.tolist()}
    

    
pd.DataFrame.from_dict(dicionario_indicadores, orient='index')

,R-Quadrado,R-Quadrado Ajustado,AIC,Coef
0.000,0.291,0.29,63533.505369,"[-2711.399448678682, 564.5779904034882, 6173.5..."
0.001,0.291,0.29,63533.505369,"[-2711.399448678682, 564.5779904034882, 6173.5..."
0.005,0.291,0.29,63533.505369,"[-2711.399448678682, 564.5779904034882, 6173.5..."
0.010,0.291,0.29,63533.505369,"[-2711.399448678682, 564.5779904034882, 6173.5..."
0.050,0.291,0.29,63533.505369,"[-2711.399448678682, 564.5779904034882, 6173.5..."
0.100,0.291,0.29,63533.505369,"[-2711.399448678682, 564.5779904034882, 6173.5..."


In [24]:
dicionario_indicadores = {}

for alpha in lista_de_alpha:
    md_treino = smf.ols(f'renda ~ {string_variaveis}', data=X_train.join(y_train))
    reg = md_treino.fit_regularized(method="elastic_net", refit=True, L1_wt=0, alpha=alpha)

    y_pred_test = reg.predict(X_test)

    r_quadrado = r2_score(y_test, y_pred_test)
    r_quadrado_ajustado = 1 - ((1 - r_quadrado) * (len(y_test) - 1)) / (len(y_test) - len(X_test.columns) - 1)
    
    rss = np.power(y_test - y_pred_test, 2).sum()
    log_vero_test = (
    -len(y_test) / 2 * (np.log(2 * np.pi) + np.log(rss/len(y_test)) + 1)
    )
    
    
    aic = 2* len(reg.params) - 2 * log_vero_test
    
    dicionario_indicadores[alpha] = {'R-Quadrado': round(r_quadrado, 3), 'R-Quadrado Ajustado': round(r_quadrado_ajustado, 3), 'AIC': aic, 'Coef': reg.params.tolist()}
    

    
pd.DataFrame.from_dict(dicionario_indicadores, orient='index')

,R-Quadrado,R-Quadrado Ajustado,AIC,Coef
0.000,0.291,0.290,63533.505369,"[-2711.3994486786796, 564.5779904034903, 6173...."
0.001,0.291,0.290,63533.263843,"[-2626.6436341440685, 564.3548656132351, 6137...."
0.005,0.291,0.290,63532.891740,"[-2326.390073636847, 563.4811849258294, 6001.2..."
0.010,0.291,0.290,63533.399984,"[-2020.840632653791, 562.4287412737425, 5845.7..."
0.050,0.287,0.286,63551.708703,"[-842.7755731642047, 555.2567809374214, 4929.9..."
0.100,0.280,0.278,63583.998565,"[-371.37376107345597, 548.4692065742388, 4176...."


### 5°

Resposta: Houve um aumento significativo no R-Quadrado ajustado dos testes, no entanto foi necessário alterar a regularização para ridge, já que lasso estava apresentando má performance na contrução do modelo, retornando valores idênticos para todos os coeficientes, AIC e etc. O melhor modelo é de alpha 0.001, avaliando R- Quadrado Ajustado e o AIC

### 6°

R-Quadrado                                                       0.291 --
R-Quadrado Ajustado                                               0.29 --
AIC                                                       63533.263843

In [25]:
string_variaveis = 'np.power(tempo_emprego,2) + sexo_M  + idade + tipo_renda_Empresário + educacao_Superior_completo'

In [26]:
md_treino = smf.ols(f"renda ~ {string_variaveis}", data=X_train.join(y_train))
reg = md_treino.fit_regularized(method="elastic_net", refit=True, L1_wt=0, alpha=0.001)

y_pred_test = reg.predict(X_test)

r_quadrado = r2_score(y_test, y_pred_test)
r_quadrado_ajustado = 1 - ((1 - r_quadrado) * (len(y_test) - 1)) / (
    len(y_test) - len(X_test.columns) - 1
)

rss = np.power(y_test - y_pred_test, 2).sum()
log_vero_test = -len(y_test) / 2 * (np.log(2 * np.pi) + np.log(rss / len(y_test)) + 1)


aic = 2 * len(reg.params) - 2 * log_vero_test

pd.DataFrame.from_dict(
    {
        "R-Quadrado": round(r_quadrado, 3),
        "R-Quadrado Ajustado": round(r_quadrado_ajustado, 3),
        "AIC": aic,
        "Coef": reg.params.tolist(),
    },
    orient="index",
)

,0
R-Quadrado,0.284
R-Quadrado Ajustado,0.283
AIC,63566.148116
Coef,"[-270.04977881447394, 19.22441260077855, 6090...."


### 7°

In [27]:
regr = DecisionTreeRegressor()
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.41025398915828437

Resposta: O modelo de árvore de regressão apresentou resultados superiores a todo o trabalho de refinamento utilizado com a função stepwise fazendo seleção de variáveis para tirar o máximo de performance da linha de regressão 